# Applied Data Analysis - Fall 2016
## Twitter-Swisscom Project

### Processing Data

This notebook contains all the operation we made on the data to prepare it for the event detection. 
Here is a summary of what we did :


1 - [Load the data](#load_data)

2 - [Remove unuseful columns](#remove_columns)

3 - [Replace Position](#replace_position)

4 - [Drop nan in position and text](#drop_nan)

5 - [Remove stopword from text](#remove_stopwords)

6 - [Get the hashtags from the text](#get_hashtags)

7 - [Save the tweets into csv for event Detection](#save_tweets)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from utils import *
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from scipy.cluster.hierarchy import ward, dendrogram,fcluster

from sklearn.cluster import DBSCAN
from sklearn import metrics

import datashader as ds
import datashader.transfer_functions as tf

### 1 -  <a id='load_data'>Load the data</a> 

We load the data for event we created at the pre-processing step.

In [ ]:
col_event_split = ['id','userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId', 'inReplyTo', 'placeLatitude', 'placeLongitude']

In [ ]:
parse_dates = ['createdAt']

In [ ]:
tweets = pd.read_csv('../twitter-swisscom/twex_event_corrected.tsv', sep="\t", encoding='utf-8', escapechar='\\', names=col_event_split, parse_dates=parse_dates, na_values='N', header=None)

In [ ]:
tweets.head()

### 2 - <a id='remove_columns'>Drop unuseful columns</a>

We will not use the 'placeId' and the 'inReplayTo' informations to do our detection, so we drop them now.

In [ ]:
tweets.drop(['placeId', 'inReplyTo'], inplace=True, axis=1)

### 3 - <a id='replace_position'>Replace Position</a>

Different informations on the longitude and latitude are given, the columns 'longitude'/'latitude' are the position of the Tweet as reported by the user or client application. The 'placeLongitude'/'placeLatitude' indicates that the tweet is associated to a place. And as we can see on the head of the table, the place is not always set.

We decided to use the longitude/latitude columns to represent the position of a tweet and if they are null we will use the placeLatitude and placeLongitude. If both are null we will have to drop the entry as a tweet without position is not usefull for event detection.

In [ ]:
tweets = tweets.apply(replace_position, axis=1)

In [ ]:
tweets.head()

We can now drop the columns 'placeLongitude' and 'placeLatitude' as they don't give us anymore informations.

In [ ]:
tweets.drop(['placeLatitude', 'placeLongitude'],inplace=True, axis=1)

### 4 - <a id='drop_nan'>Drop nan in position and text</a>

Then we drop the NaN values in Longitude and Latitude columns as we need a position to detect event.

In [ ]:
len_before = len(tweets.index)
tweets = tweets.dropna(subset=['longitude', 'latitude'])
len_after = len(tweets.index)
print("Number of tweets before dropping the one without position : ", len_before)
print("Number of tweets before dropping the one without position : ", len_after)
print("Percentage of tweets lost : ", ((len_before - len_after)/len_before)*100)

As we also base our event detection on the text field we don't want to have nan value in it. So we drop them.

In [ ]:
len_before = len(tweets.index)
tweets = tweets.dropna(subset=['text'])
len_after = len(tweets.index)
print("Number of tweets before dropping the one without text : ", len_before)
print("Number of tweets before dropping the one without text : ", len_after)
print("Percentage of tweets lost : ", ((len_before - len_after)/len_before)*100)

We decide to detect an event by its day of occurence we create a new column that gives us the information of the day.

In [ ]:
tweets['day'] = pd.DatetimeIndex(tweets['createdAt']).normalize()

### 5 - <a id='remove_stopwords'>Remove stopword from the text field</a>

We decide to remove the stopwords from the tweets' text to keep only words that can describe an event.

In [ ]:
stop_words = stopwords.words('english')
stop_words += stopwords.words('french')
stop_words += stopwords.words('german')
stop_words += stopwords.words('italian')
stop_words += string.punctuation
stop_words += ['—','/via','via', 'follow', 'please', 'i\'m', '^_^', ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
':c', ':{', '>:\\', ';(', ':-)', ':)', ';)','[=o)]', ';-)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
'=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
'<3']

In [ ]:
def text_process(row):
    text = row['text']
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@ \S+", "", text)
    text = re.sub(r"@\S+", "", text)
    text = text.split()
    text  = [word for word in text if word.lower() not in stop_words]
    row['text'] = text
    return row

We remove the stopwords but we also remove the URLs and the @ mentions as they are not useful to detect events

In [ ]:
tweets = tweets.apply(text_process, axis=1)
tweets.head()

### 6 - <a id='get_hashtags'>Get the hashtags from the text</a>

Now that the text is pretty much clean we want to get the hashtags from the tweets because they are really helpful to detect the events. So we create a column with the hashtags :

In [ ]:
tweets['hashtags'] = tweets['text'].apply(find_hashtags)

And in the "text" fields we don't want to have '#' anymore.

In [ ]:
tweets['text'] = tweets['text'].apply(remove_hashtags)

### 7 - <a id='save_tweets'>Save the tweets into csv for event Detection</a>

We save the current dataframe so we don't need to re-run the preprocessing everytime as it takes a lot of time.

In [ ]:
tweets.to_csv("../twitter-swisscom/event/tweets_processed.csv", sep=',', encoding='utf-8', index=False)

## EVENT  DETECTION

Now that our dataframe is clean we want to find the event hidden in all those tweets. Here are our assumptions to find an event :
- An event is described by a hashtags it contains
- An event takes place at a certain place described by longitude/latitude (we took a radius of 3km)
- An event occurs during a certain  time (we decided to find them day by day)
- An event has a minimum number of people talking about it (we took 20) and a minimum of tweets (we took 50)

We load the csv file containing our tweets preprocessed.

In [2]:
tweets = pd.read_csv("../twitter-swisscom/twex_event_tweets_processed.csv", sep=',', encoding='utf-8')

As we want to detect event day per day we will have to go through all the day in our data and for each of them detect the events.

In [3]:
days = np.unique(tweets['day'].values)
print(len(days))

2215


### Detection on one day

Before applying the detection on the whole data, we want to do the detection on a specific day so that the pipeline is easy to understand.

In [ ]:
from random import randint
index = randint(0,len(days))

day = tweets[tweets['day']==days[2084]]

We use [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) to detect clusters of tweets that are close to each others. And we use this cluster assignment to detect events.

In [ ]:
X = np.radians(day.as_matrix(columns=['latitude', 'longitude']))
db = DBSCAN(eps=1/6371, min_samples=5, metric='haversine').fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
day = day.assign(cluster=db.labels_)

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print('Estimated number of clusters: %d' % n_clusters_)

We print the clusters position for this given day.

In [ ]:
unique_labels = set(labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
for k, col in zip(unique_labels, colors):
    if k == -1:
        col = 'k'

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
df_event_paleo = pd.DataFrame(columns=['name', 'date', 'keywords', 'hashtags', 'longitude', 'latitude', '# of tweets', '# of people', 'tweetids'])

In [ ]:
day_df = tweets[tweets['day']=="2016-07-20 00:00:00"]
    
X = np.radians(day_df.as_matrix(columns=['latitude', 'longitude']))
db = DBSCAN(eps=3/6371, min_samples=5, algorithm="ball_tree", metric='haversine').fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

day_df = day_df.assign(cluster=db.labels_)

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

for cluster in range(0,n_clusters_-1):
    ntweets = len(day_df[day_df['cluster']==cluster])
    npeople = len(np.unique(day_df[day_df['cluster']==cluster].userId))
    ids = list(day_df[day_df['cluster']==cluster].id.values)
    d_hashtags = dictionnary_from_hashtags(day_df, cluster)
    d_keywords = dictionnary_from_keywords(day_df, cluster)

    d_hashtags_detection = useful_(d_hashtags, 1/10, ntweets)
    d_keywords_detection = useful_(d_keywords, 1/10, ntweets)

    if(any(d_hashtags_detection)):
        position = find_position(day_df, cluster, ntweets)
        i = len(df_event_paleo)
        name = ""
        for elem in d_hashtags_detection.keys(): 
            name += elem + " "
        df_event_paleo.loc[i] = ([name[:-1], "2016-07-19 00:00:00", list(d_keywords_detection.keys()), list(d_hashtags_detection.keys()), position[0], position[1], ntweets, npeople, ids])
        #print(" ------- Event added -------")

In [ ]:
len(df_event_paleo)

## Create the dataframe containing the events

We go through the clusters and find out if the corresponding tweets are linked by an event or not. To do so we first of all create a dataframe that will contain the events. (Name, date, keywords, hashtags, longitude, latitude, #of tweets, #of people, tweetids)

In [4]:
df_event = pd.DataFrame(columns=['name', 'date', 'keywords', 'hashtags', 'longitude', 'latitude', '# of tweets', '# of people', 'tweetids'])

Now we want to populate our dataframe with real event. So we go through all the clusters and find out the hashtags, keywords and corresponding event based on these two infos. We use the same pipeline we used above to detect the clusters.

In [5]:
tot = len(days)
iter_=0
for day in days:
    if(iter_%50==0):
        print(str(iter_) + '/' + str(tot))
    iter_+=1
    day_df = tweets[tweets['day']==day]
    
    X = np.radians(day_df.as_matrix(columns=['latitude', 'longitude']))
    db = DBSCAN(eps=3/6371, min_samples=5, algorithm="ball_tree", metric='haversine').fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    day_df = day_df.assign(cluster=db.labels_)

    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    for cluster in range(0,n_clusters_-1):
        ntweets = len(day_df[day_df['cluster']==cluster])
        npeople = len(np.unique(day_df[day_df['cluster']==cluster].userId))
        if(ntweets>=50 & npeople>=20): 
            ids = list(day_df[day_df['cluster']==cluster].id.values)
            d_hashtags = dictionnary_from_hashtags(day_df, cluster)
            d_keywords = dictionnary_from_keywords(day_df, cluster)

            d_hashtags_detection = useful_(d_hashtags, 1/10, ntweets)
            d_keywords_detection = useful_(d_keywords, 1/10, ntweets)

            if(any(d_hashtags_detection)):
                position = find_position(day_df, cluster, ntweets)
                i = len(df_event)
                name = ""
                for elem in d_hashtags_detection.keys(): 
                    name += elem + " "
                df_event.loc[i] = ([name[:-1], day, list(d_keywords_detection.keys()), list(d_hashtags_detection.keys()), position[0], position[1], ntweets, npeople, ids])

0/2215
50/2215
100/2215
150/2215
200/2215
250/2215
300/2215
350/2215
400/2215
450/2215
500/2215
550/2215
600/2215
650/2215
700/2215
750/2215
800/2215
850/2215
900/2215
950/2215
1000/2215
1050/2215
1100/2215
1150/2215
1200/2215
1250/2215
1300/2215
1350/2215
1400/2215
1450/2215
1500/2215
1550/2215
1600/2215
1650/2215
1700/2215
1750/2215
1800/2215
1850/2215
1900/2215
1950/2215
2000/2215
2050/2215
2100/2215
2150/2215
2200/2215


In [6]:
len_event_no_filter = len(df_event)
print("Number of event without filtering the spam neither the event not in the scope : ", len_event_no_filter)

Number of event without filtering the spam neither the event not in the scope :  1181


We want to avoid having spam event in our event dataframe. Here are the 3 assumptions we did to avoid the spam events :
- (number of tweets) / (number of people) has to be less than 6.
- number of people involved needs to be greater or equal to 20.
- number of tweets per event needs to be greater or equal to 50.

In [7]:
df_event_nofilter = df_event

In [ ]:
df_event = df_event[df_event["# of tweets"]/df_event["# of people"] < 6]

In [ ]:
df_event = df_event[(df_event["# of people"] >= 20) & (df_event["# of tweets"] >= 50)]

In [ ]:
len_event_no_spam = len(df_event)
print("Number of event after filtering the spam : ", len_event_no_spam)

Our dataset should be containing tweets between 2010 and 2016 but we still want to make sure we don't have outliers so we filter it.

In [8]:
df_event['date'] = pd.DatetimeIndex(df_event['date']).normalize()

In [9]:
df_event = df_event[(df_event.date >= "2010-01-01") & (df_event.date <= "2016-12-31")]

We save our dataframe to find the countries linked with each event.

In [10]:
df_event.to_csv("../twitter-swisscom/event_no_country.csv", sep=',', encoding='utf-8', index=False)

Here is the pipeline we used to find the country given the longitude/latitude of an event :
- Install OSGeo4W : https://trac.osgeo.org/osgeo4w/wiki
- Open the OSGeo4W shell
- Go to the directory where our script is (/country-detection)
- install pandas : pip install pandas
- run the script : python find_country.py



(in our script we use these two tools : https://github.com/che0/countries)

We then reload the dataframe with the countries and remove the one that are not in Switzerland

In [11]:
df_event = pd.read_csv("../twitter-swisscom/event_country.csv", sep=',', encoding='utf-8')

In [12]:
len_total = len(df_event)
df_event = df_event[df_event.country == 'CH']
len_swiss = len(df_event)
print("Total number of events : ", len_total)
print("Number of events in Switzerland : ", len_swiss)
print("Loss percentage : ", (1-(len_swiss/len_total))*100, "%")

Total number of events :  1181
Number of events in Switzerland :  845
Loss percentage :  28.450465707027938 %


In [13]:
len_event_final = len(df_event)
print("Number of event after filtering the event not in the scope (temporal and spatial) : ", len_event_final)

Number of event after filtering the event not in the scope (temporal and spatial) :  845


In [14]:
df_event.head()

,name,date,keywords,hashtags,longitude,latitude,# of tweets,# of people,tweetids,country
0,iPad2 applestoreZH,2011-03-25,"['iPad2', 'applestoreZH']","['iPad2', 'applestoreZH']",8.534900,47.376152,119.0,35.0,"[51171739434106880, 51172379367448576, 5117898...",CH
1,emex11,2011-08-24,['emex11'],['emex11'],8.539642,47.388574,99.0,33.0,"[106229984795963393, 106233842423177216, 10623...",CH
2,blw5,2011-09-01,['blw5'],['blw5'],8.530394,47.379191,67.0,36.0,"[109138385737756672, 109143075309359105, 10915...",CH
3,fec11,2011-09-09,['fec11'],['fec11'],8.538244,47.391208,106.0,39.0,"[112043612723875840, 112044836349153280, 11204...",CH
4,InnoLab,2011-09-29,['InnoLab'],['InnoLab'],8.543167,47.386950,64.0,32.0,"[119238375667548160, 119282167158276096, 11929...",CH


We save our dataframe to a csv as all the detection is done and we want to use it for our visualization now.

In [15]:
df_event.to_csv("../twitter-swisscom/event_final_events.csv", sep=',', encoding='utf-8', index=False)

## After visualization correction

We reload our csv and make some tweaks to make our detection more precise.

In [16]:
df_event = pd.read_csv("../twitter-swisscom/event_final_events.csv", sep=',', encoding='utf-8')

In [17]:
df_event.head()

,name,date,keywords,hashtags,longitude,latitude,# of tweets,# of people,tweetids,country
0,iPad2 applestoreZH,2011-03-25,"['iPad2', 'applestoreZH']","['iPad2', 'applestoreZH']",8.534900,47.376152,119.0,35.0,"[51171739434106880, 51172379367448576, 5117898...",CH
1,emex11,2011-08-24,['emex11'],['emex11'],8.539642,47.388574,99.0,33.0,"[106229984795963393, 106233842423177216, 10623...",CH
2,blw5,2011-09-01,['blw5'],['blw5'],8.530394,47.379191,67.0,36.0,"[109138385737756672, 109143075309359105, 10915...",CH
3,fec11,2011-09-09,['fec11'],['fec11'],8.538244,47.391208,106.0,39.0,"[112043612723875840, 112044836349153280, 11204...",CH
4,InnoLab,2011-09-29,['InnoLab'],['InnoLab'],8.543167,47.386950,64.0,32.0,"[119238375667548160, 119282167158276096, 11929...",CH


After vizualizing our data on several years, we see that some events seems to not be spatial but only like trending hashtags, we can detect them by finding events that take place at the same time but in many different places. We don't want to show them on the map as there position is not really representative.

In [18]:
df_event['trending'] = "False"

In [19]:
trending_event = df_event.groupby(['name','date'])

In [20]:
namelist = []
def getnames(group):
    name = group.name
    if len(group['longitude'])>1:
        namelist.append(name[0])

trending_event.apply(getnames)

""


In [21]:
namelist

['1DCH',
 'CanYouMakeIt16',
 'EMABiggestFans1D',
 'MTVHottest',
 'brw15',
 'christmasrocks',
 'christmasrocks',
 'christmasrocks',
 'justloveit',
 'newyearrocks']

In [22]:
df_event['trending'] = df_event['name'].isin(namelist)

In [23]:
len(df_event[df_event['trending']==True])

152

We also found out that some events were duplicated because they are were taking place on different following days. We don't want to make two different bubbles on our vizualization for the same event, so we will group them. We do it in our vizualization.

In [24]:
len(df_event)

845

Our final event dataframe is saved and used in the viz.

In [25]:
df_event.to_csv("../twitter-swisscom/events_detected_final.csv", sep=',', encoding='utf-8', index=False)